<a href="https://colab.research.google.com/github/bramosguevara/AlgoritmosdeOptimizacion/blob/main/TrabajoPr%C3%A1ctico_AlgoritmosOptimizacion_BriamSebastianRamosGuevara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: **Briam Sebastian Ramos Guevara**  <br>
Url: https://github.com/bramosguevara/AlgoritmosdeOptimizacion <br>
Google Colab: https://colab.research.google.com/drive/11UzPcKaWX2as0qqErwIQHdg5jYhhD8vJ <br>
Problema:
>2. Organizar los horarios de partidos de una jornada de La Liga<br>

Descripción del problema: **Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la
audiencia**                             

**Solucion al problema:**
Se implementa la optimización en función de maximizar la audiencia mediante un algoritmo genético el cual se basa en varios factores clave como lo son:
- La optimizacion en un espacio de busqueda complejo
- Evita optimos locales con diversidad genetica
- La incorporacion de restricciones y penalizaciones
- Busqueda local memética para el refinamiento adicional
- Flexibilidad y paralelización


#Modelo
- **¿Como represento el espacio de soluciones?**

El espacio de soluciones esta representado mediante cromosomas, el cual esta conformado por todas las posibles combinaciones de horarios para los partidos. En este caso cada cromosoma se compone de una lista de tamaño n = 10 indicando la cantidad de partidos. A su vez el gen en el cromosoma representa el horario asignado a un partido especifico. Asi mismo, se implementa que los valores de los genes son numeros enteros en el rango de 0 a 9 en el cual cada uno de ellos representa cada franja horaria.

- **¿Cual es la función objetivo?**

En este problema la funcion objetivo es maximizar la audiencia total de todos los partidos. Esto se calcula mediante la funcion calcular_fitness(), en la cual se evalua un cromosoma tal que:
- Tenga una audiencia base, en torno a las categorias de los equipos enfrentados
- Ajuste la audiencia en funcion de la hora en que se transmite el partido
- En caso de que no haya exactamente un partido el viernes y otro el lunes, se aplique una penalización severa.

- **¿Como implemento las restricciones?**

En este problema se puede tener en cuenta dos tipos de restricciones, una en cuanto a la restricción en la generación de individuos la cual se denota por la función generar_individuo(), donde se asegura que haya un partido un viernes y haya un partido el lunes y los demas partidos se asignen entre sabado y domingo. <br>
Por otro lado, se tiene la restricción en la evaluación de las soluciones en la cual durante la evaluación de cada cromosoma en la función calcular_fitness(), se aplican penalizaciones si no se cumplen ciertas reglas como: Si hay muchos partidos en el mismo horario, la audiencia de cada uno se reduce con un factor de penalización (factor_coincidencia). Y también, si un cromosoma no tiene un partido el lunes y otro el viernes, la funcion se reduce drasticamente.

In [1]:
import random

# PARÁMETROS DEL GA

POPULATION_SIZE = 50     # Tamaño de población
N_GENERATIONS   = 200    # Número de generaciones
P_CROSSOVER     = 0.8    # Probabilidad de cruce
P_MUTATION      = 0.2    # Probabilidad de mutación
TOURNAMENT_SIZE = 3      # Tamaño torneo (selección)
LS_PROB         = 0.3    # Probabilidad de búsqueda local en hijos

random.seed(42)  # Semilla fija para reproducibilidad

# Datos del problema

partidos = [
    ('B','A'),  # Celta - Real Madrid
    ('B','A'),  # Valencia - R. Sociedad
    ('C','C'),  # Mallorca - Eibar
    ('B','A'),  # Athletic - Barcelona
    ('C','C'),  # Leganés - Osasuna
    ('B','C'),  # Villarreal - Granada
    ('B','B'),  # Alavés - Levante
    ('B','B'),  # Espanyol - Sevilla
    ('B','C'),  # Betis - Valladolid
    ('B','B'),  # Atlético - Getafe
]
N = len(partidos)  # Número de partidos (10)

# 1) Audiencia base según las categorías de equipos
audiencia_base = {
    ('A','A'): 2.0,   ('A','B'): 1.3,   ('A','C'): 1.0,
    ('B','A'): 1.3,   ('B','B'): 0.9,   ('B','C'): 0.75,
    ('C','A'): 1.0,   ('C','B'): 0.75,  ('C','C'): 0.47
}

# 2) Factores de horario (ajuste de audiencia según la hora)
coef_horario = {
    0: 0.40,  1: 0.55,  2: 0.70,  3: 0.80,  4: 1.00,
    5: 0.45,  6: 0.75,  7: 0.85,  8: 1.00,  9: 0.40
}

# 3) Penalización por coincidencias (reducción de audiencia)
reduccion_coinc = [0.00, 0.25, 0.45, 0.60, 0.70, 0.75, 0.78, 0.80, 0.80]

def factor_coincidencia(k_menos_1):
    if k_menos_1 < 0:
        return 1.0
    if k_menos_1 >= len(reduccion_coinc):
        return 1.0 - 0.80  # Penalización máxima
    return 1.0 - reduccion_coinc[k_menos_1]

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Se tiene que para este algoritmo se presenta un orden de complejidad pseudopolinomial: O(N_GENERATIONS⋅POPULATION_SIZE⋅N), lo cual es manejable y adecuado para este problema, ya que, al tener parametros realistas el algoritmo converge a una buena solucion en un tiempo razonable.

Se puede obtener el calculo de espacio de soluciones donde se tiene que:

- Sin restricciones se llega a $10^{10}$ combinaciones.
- Con restricciones se llega alrededor de 756 millones de combinaciones.
- Como el algoritmo genetico evalua population_size soluciones por n_generations, donde en este caso equivale a population_size=50 y n_generations=200, el numero total de evaluaciones de fitness es: 10000 <br>
Esto quiere decir que el algoritmo genetico, no requiere abordar todas las posibles combinaciones sino que con una fraccion del espacio total es capaz de encontrar soluciones optimas.

In [2]:

# FUNCIÓN DE APTITUD (Evaluación de soluciones)

def calcular_fitness(cromosoma):
    conteo_slots = [0]*10
    total_audiencia = 0.0

    for slot in cromosoma:
        conteo_slots[slot] += 1

    for i, slot in enumerate(cromosoma):
        cat1, cat2 = partidos[i]
        base = audiencia_base[(cat1, cat2)]
        ch = coef_horario[slot]  # factor horario
        k = conteo_slots[slot]   # número de partidos en el mismo horario
        fc = factor_coincidencia(k-1)

        total_audiencia += (base * ch * fc)

    # Restricción: Debe haber un partido en viernes y otro en lunes
    if conteo_slots[0] != 1 or conteo_slots[9] != 1:
        total_audiencia *= 0.01  # Penalización fuerte

    return total_audiencia



# GENERACIÓN DE INDIVIDUOS (Población inicial)

def generar_individuo():
    crom = [None]*N
    idx_viernes = random.randrange(N)
    idx_lunes = idx_viernes
    while idx_lunes == idx_viernes:
        idx_lunes = random.randrange(N)

    crom[idx_viernes] = 0  # Viernes
    crom[idx_lunes] = 9  # Lunes

    for i in range(N):
        if i not in (idx_viernes, idx_lunes):
            crom[i] = random.randint(1,8)  # Sábado o domingo

    return crom

def generar_poblacion(size=POPULATION_SIZE):
    return [generar_individuo() for _ in range(size)]



# OPERADORES GENÉTICOS

def torneo_seleccion(poblacion, fitnesses, k=TOURNAMENT_SIZE):
    elegidos = random.sample(list(zip(poblacion, fitnesses)), k)
    elegidos.sort(key=lambda x: x[1], reverse=True)
    return elegidos[0][0]

def cruzar_padres(p1, p2):
    punto = random.randint(1, N-1)
    h1 = p1[:punto] + p2[punto:]
    h2 = p2[:punto] + p1[punto:]
    return h1, h2

def mutar(cromosoma):
    idx_viernes = [i for i,s in enumerate(cromosoma) if s == 0]
    idx_lunes = [i for i,s in enumerate(cromosoma) if s == 9]

    for i in range(N):
        if i in idx_viernes or i in idx_lunes:
            continue
        if random.random() < P_MUTATION:
            cromosoma[i] = random.randint(1,8)  # Cambio de horario aleatorio
    return cromosoma



# BÚSQUEDA LOCAL

def busqueda_local(cromosoma):
    best_crom = cromosoma[:]
    best_fit = calcular_fitness(best_crom)

    idx_viernes = [i for i,s in enumerate(cromosoma) if s == 0]
    idx_lunes = [i for i,s in enumerate(cromosoma) if s == 9]

    for i in range(N):
        if i in idx_viernes or i in idx_lunes:
            continue
        slot_actual = best_crom[i]
        for slot_posible in range(1,9):
            if slot_posible == slot_actual:
                continue
            temp = best_crom[:]
            temp[i] = slot_posible
            fit_temp = calcular_fitness(temp)
            if fit_temp > best_fit:
                best_fit = fit_temp
                best_crom = temp
    return best_crom

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Para resolver el problema se utilizó la técnica de algoritmos genéticos, donde se encarga de asignar horarios a los partidos de forma que la audiencia total sea la máxima posible, considerando las restricciones como: Horarios con diferentes niveles de audiencia, penalización por coincidencias de partidos en el mismo horario, restricción debido a que un partido deba jugarse el lunes y otro el viernes.

Se escoge esta técnica en comparación con otras, en pro de maximizar los beneficios que brinda esta, debido a que el problema tiene un espacio de soluciones muy grande, es un problema de optimización combinatoria, así mismo la función objetivo es compleja y no se tiene una forma clara de análisis de la misma, también permite el manejo de las restricciones de forma más flexible, se tiene en cuenta una búsqueda global y la exploración de múltiples soluciones lo que nos permite estar tranquilos de no caer en óptimos locales, y también muy importante la escalabilidad y adaptabilidad de la optimización en el cual se pueden ajustar los parámetros para tener un mejor balance entre la explotación y exploración.

Gracias a estas características, para este problema el algoritmo genético es un buen mecanismo y muy capaz para encontrar la solución al encontrar una buena asignación de horarios para maximizar la audiencia en un tiempo razonable, comparado con otras técnicas como búsqueda exhaustiva donde resulta casi imposible llegar a una solución como la anterior.

In [3]:
# ALGORITMO GENÉTICO PRINCIPAL

def algoritmo_genetico():
    poblacion = generar_poblacion()

    for gen in range(N_GENERATIONS):
        fitnesses = [calcular_fitness(ind) for ind in poblacion]
        nueva_pop = []

        elite = sorted(zip(poblacion, fitnesses), key=lambda x:x[1], reverse=True)
        nueva_pop.append(elite[0][0][:])
        nueva_pop.append(elite[1][0][:])

        while len(nueva_pop) < POPULATION_SIZE:
            p1 = torneo_seleccion(poblacion, fitnesses, TOURNAMENT_SIZE)
            p2 = torneo_seleccion(poblacion, fitnesses, TOURNAMENT_SIZE)

            h1, h2 = cruzar_padres(p1, p2) if random.random() < P_CROSSOVER else (p1[:], p2[:])
            h1, h2 = mutar(h1), mutar(h2)

            if random.random() < LS_PROB:
                h1 = busqueda_local(h1)
            if random.random() < LS_PROB:
                h2 = busqueda_local(h2)

            nueva_pop.append(h1)
            if len(nueva_pop) < POPULATION_SIZE:
                nueva_pop.append(h2)

        poblacion = nueva_pop

    fitnesses = [calcular_fitness(ind) for ind in poblacion]
    best_index = max(range(len(poblacion)), key=lambda i: fitnesses[i])
    return poblacion[best_index], fitnesses[best_index]

# EJECUCIÓN PRINCIPAL
if __name__ == "__main__":
    sol, fit = algoritmo_genetico()
    print("Mejor asignación encontrada:", sol)
    print("Audiencia total (fitness):", round(fit, 2))

Mejor asignación encontrada: [8, 4, 0, 7, 9, 1, 2, 6, 5, 3]
Audiencia total (fitness): 6.86
